# Como usar

Esse aplicativo web é divido em 3 partes:
1. **Gerar o seu sistema:** Isso pode ser feito após entrar as informações do domínio da função de onda $\Psi(x)$, o número de pontos utilizados para representá-la, a forma da **energia potencial $V(x)$** e clicar em `Gerar`.
    * Existem energias potenciais $V(x)$ pré definidas dentro do menu `Potenciais pré definidos`. Se escolher uma opção pré definida, o domínio e o número de pontos serão selecionados automaticamente e, em seguida, o botão 'Gerar' será clicado.
    * A expressão para $V(x)$ está em eletron-volts, e $x$ estará em unidades Raio de Bohr (veja sobre as unidades de Rydberg abaixo).
    * Caso você deseje utilizar $x$ em Angstroms, divida-o sempre por `ang`. Por exemplo:
        * **$V(x)=((x/ang) - 10)^2$ resultará em um potencial harmônico em eletron-volts, centrado em $x=10$ Angstroms.**
        * **$V(x)=(x - 10)^2$ resultará em um potencial harmônico em eletron-volts, centrado em $x=10$ Raio de Bohr.**


2. **Diagonalizar a matriz do sistema:** Após o sistema ser gerado, o menu `Diagonalizar/Informações` exibirá uma tabela com algumas informações sobre o sistema gerado. Se estiver tudo como você quer, clique em `Diagonalizar` para resolver a matriz correspondente ao sistema, gerando as auto funções e os auto valores.

3. **Plotar o resultado:** Após exibida a mensagem `Diagonalizado!` no botão `Diagonalizar`, siga para a aba `Funções de onda`, onde você poderá clicar em 'Plotar' para plotar os gráficos correspondentes das soluções do sistema escolhido.

***

In [3]:
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt
from numba import jit
import ipywidgets as wid
from ipywidgets import interactive,fixed, interactive_output

RY=13.60566 #x*RY: Rydberg -> ev
ang=1.88972 #X/ang: raio de bohr -> angstroms
pot_exemplos=['(2e-4)*((x)**2)','0',
             '(1e-5)*(x**2)+(1e-7)*(x**4)',
             '(10)*( (49.2 / (x/ang+84) )**12 - (49.2 / (x/ang+84) )**6)',
             '3.5*(np.exp(-2*(0.1*(x/ang) + 2.75)) - 2*np.exp(-(0.1*(x/ang) + 2.75)))',
             '-4*(np.cosh(0.1*x/ang)**(-2))',
             '0.05*abs(x/ang)-1.5',
             '(1e-5)*(x**2)-(2e-7)*(x**4)',
             '0.05*x/ang',
             '-0.05*x/ang']
def diagg(boo):
    global x,psi,prob,eig,V,L,N
    DIAG.description='Diagonalizando...'
    
    @jit
    def diagH(H,V):
        H=H+V
        eig, psi=np.linalg.eigh(H) #Resolve o sistema e atribui-se: eig=autovalores, psi=autovetores
        prob=np.abs(psi)**2 #Atribui a densidade de probabilidade |Psi|² a variável prob
        return H,V,psi,prob,eig
    
    N=x.size #Armazena o tamanho do vetor posição e o comprimento do espaço que ele representa
    a=- 1/(dx**2) #Define 'a' por conveniência
    H=sparse.diags([a,-2*a,a],[-1,0,1],shape=(N,N),dtype=np.complex128).toarray() #Cria a matriz Hamiltoniano
    H,V,psi,prob,eig=diagH(H,V)
    
    V=np.diag(V)
    PLOT.children[8].disabled=False
    PLOT.children[8].description=('Plotar')
    PLOT.children[8].click()
    DIAG.disabled=True
    DIAG.description='Diagonalizado!'
    tab.selected_index=1
    
def syst(xx,N,potV):   
    global x,dx,V,Vlim
    def vetores(xx,N):
        xmin,xmax=xx
        x,dx=np.linspace(xmin*ang,xmax*ang, N, 
                         retstep=True,dtype=np.float64) #Define o vetor posição (raio de bohr)
        return x,dx,xmin,xmax
    
    def pot(N,pot):
        V=np.ones(shape=(N))
        V=V*eval(pot)/RY
        V=np.diag(V)
        return V
    
    x, dx, xmin, xmax=vetores(xx,N)
    V=pot(N,potV)
    
    xmin, xmax= x.min()/ang, x.max()/ang
    L=xmax-xmin
    Vlim=[V.min()*RY, V.max()*RY]
    info.children[1].value=f"{dx/ang: .2f}"
    info.children[3].value=f"{L: .2f}"
    info.children[5].value=f"[{Vlim[0]:.5f} , {Vlim[1]:.5f}]"
    info.children[7].value=SYST.children[2].value
    info.children[9].value=f"{SYST.children[1].value: 1d}"
    
    fig1, ax1= plt.subplots(figsize=(12,8))
    ax1.set_xlabel('$x (\AA)$')
    ax1.set_ylabel('Potencial (eV)')  
    
    plt.plot(x/ang,np.diag(V)*RY,'r-',label='$V(x)$')    
    plt.axhline(Vlim[0], xmin, xmax,color='r',ls='--',label='limites de $V(x)$')
    plt.axhline(Vlim[1], xmin, xmax,color='r',ls='--')
    plt.axvline(x=x.min()/ang,color='k',ls='--',label='limites de $x$')
    plt.axvline(x=x.max()/ang,color='k',ls='--',)
    plt.setp(ax1, xticks=[xmin, xmax], yticks=Vlim)
    leg=plt.legend(loc='upper right')
    
    DIAG.disabled=False
    DIAG.description='Diagonalizar'
    PLOT.children[8].disabled=True
    PLOT.children[8].description=('Diagonalize...')
    INFO.selected_index=0
    return V,x,dx,xmin,xmax
    
def plotall(NES,limx,wav,xs,ys,dpi,limy,autolim):
    fig, ax = plt.subplots(figsize=(xs,ys), dpi=dpi)
    nmin,nmax = NES
    En=eig*RY
    #elim=(En.min(), En.max())###########
    ax.set_xlim([x.min()/ang-limx, x.max()/ang +limx])
    ax.set_xlabel('$x (\AA)$')
    ax.set_ylabel('Auto energias (eV)')
    plt.title(f"$\Delta x$ = {dx/ang:.2f}, Ordem da matriz = {N} x {N}")
    probb=norm(prob,nmin,nmax,En,1)
    if wav==1:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,probb[:,i]+En[i], 
                         label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
    elif wav==2:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,np.real(psi[:,i])+En[i], 
                         label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
    elif wav==3:
        for i in range(nmax,nmin-1,-1):
            fig=plt.plot(x/ang,np.imag(psi[:,i])+En[i], 
                         label=f'$E_{{{i}}}={En[i]:.5f}$ eV')
    if autolim==False:
        ax.set_ylim(limy)
        
    fig=plt.axvline(x=x.min()/ang,color='k',ls='--')
    fig=plt.axvline(x=x.max()/ang,color='k',ls='--')
    fig=plt.plot(x/ang,V*RY,'k',label='V($x$)')
    leg=plt.legend(loc='upper right')
    
    plt.show()

@jit
def norm(prob,nmin,nmax,En,wav):
    mmax=np.zeros(nmax+2)
    if wav==1:
        for i in range(0,nmax+2):
            mmax[i]=(En[i+1]-En[i])*0.9
        for i in range(0,nmax+2):
            prob[:,i]=prob[:,i]*np.abs(mmax[i]/prob[:,i].max())
    else:
        for i in range(1,nmax+2):
            mmax[i]=(prob[:,i+1].min()-prob[:,i-1].max() \
                    /(prob[:,i].max()-prob[:,i].min()))
        for i in range(1,nmax+2):
            prob[:,i-1]=prob[:,i-1]*np.abs(mmax[i]*0.9)
    return prob

def potlist(bo):
    b=bo.description
    if b=='Oscilador Harmônico':
        SYST.children[2].value=pot_exemplos[0]
    elif b=='Poço Infinito':
        SYST.children[2].value=pot_exemplos[1]
    elif b=='Duffing Holmes (+)':
        SYST.children[2].value=pot_exemplos[2]
    elif b=='Lennard Jones':
        SYST.children[2].value=pot_exemplos[3]
    elif b=='Morse':
        SYST.children[2].value=pot_exemplos[4]
    elif b=='Pöschl–Teller':
        SYST.children[2].value=pot_exemplos[5]
    elif b=='Poço Triangular':
        SYST.children[2].value=pot_exemplos[6]
    elif b=='Duffing Holmes (-)':
        SYST.children[2].value=pot_exemplos[7]
    elif b=='Campo Uniforme (-)':
        SYST.children[2].value=pot_exemplos[8]
    elif b=='Campo Uniforme (+)':
        SYST.children[2].value=pot_exemplos[9]
    SYST.children[0].value=[-35,35]
    SYST.children[3].click()
    
################################STYLES/LAYOUT###############################
lslider=wid.Layout(width='90%', display='flex')
sty = {'description_width': 'initial'}


################################INTERACTIVES################################
SYST=interactive(syst, {'manual': True}, 
                    xx=wid.FloatRangeSlider(
                        value=[-35.0, 35.0],
                        min=-150.0,
                        max=150.0,
                        step=1,
                        continuous_update=False,
                        orientation='horizontal',
                        readout_format='1d',
                        layout=lslider,
                        description=('Domínio de $\Psi(x)$: '),
                        style=sty),
                    N=wid.IntSlider(value=550,
                        min=50,
                        max=1050,
                        step=10,
                        continuous_update=False, 
                        layout=lslider,
                        description=('Número de pontos: '),
                        style=sty),
                    potV=wid.Text(description='Expressão de $V(x)$: ',
                        value="0",
                        placeholder='Escreva a expressão aqui.',
                        style=sty,
                        layout=lslider))

PLOT=interactive(plotall, {'manual': True},
                    NES=wid.IntRangeSlider(
                        value=[0, 10],
                        min=0,
                        max=20,
                        step=1,
                        description='Plotar estados:',
                        continuous_update=False,
                        orientation='horizontal',
                        layout=lslider,
                        style=sty),
                    limx=wid.IntSlider(
                        value=20,
                        min=0,
                        max=100,
                        step=1,
                        description='Margens horizontais  (angstroms):   ',
                        continuous_update=False,
                        layout=lslider,
                        style=sty),
                    wav=wid.Dropdown(
                        options=[('densidades de probabilidades',1),
                                 ('funções de onda (parte real, +)',2),
                                 ('funções de onda (parte imaginária)',3)],
                        value=1,
                        description='Plotar as',
                        style=sty),
                    xs=wid.IntSlider(value=16,
                        min=0,
                        max=20,
                        step=1,
                        description='Largura da imagem (polegadas):',
                        style=sty,
                        layout=lslider),
                    ys=wid.IntSlider(value=9,
                        min=0,
                        max=20,
                        step=1,
                        description='Altura da imagem (polegadas): ',
                        style=sty,
                        layout=lslider),
                    dpi=wid.IntSlider(value=80,
                        min=10,
                        max=300,
                        step=1,
                        description='Pixels por polegada:',
                        style=sty,layout=lslider),
                    limy=wid.FloatRangeSlider(value=[0.,5.],
                        min=-25.,
                        max=25.,
                        step=0.05,
                        description='Limites do eixo y (angstroms):',
                        orientation='horizontal',
                        readout_format='.1f',
                        layout=lslider,
                        style=sty),
                    autolim=wid.Checkbox(
                        value=True,
                        description='Auto limitar eixo y',
                        style=sty))

################################GRIDS################################
items = [wid.Label('Passo $\Delta x $ (angstroms) ='), wid.Label('dx'), 
        wid.Label('Largura do domínio (angstroms) ='), wid.Label('L'),
        wid.Label('V(x)[min,max] (eV) ='), wid.Label('[V_min,V_max]'),
        wid.Label('Expressão V(x) ='), wid.Label(''),
        wid.Label('Número de pontos ='), wid.Label('')]
info=wid.GridBox(items, layout=wid.Layout(grid_template_columns="repeat(2, 250px)"))

################################BOTOES################################
DIAG=wid.Button(description='Diagonalizar',disabled=True)
OHS=wid.Button(description='Oscilador Harmônico')
POCO=wid.Button(description='Poço Infinito')
DUFHOLMES=wid.Button(description='Duffing Holmes (+)', style=sty)
MDUFHOLMES=wid.Button(description='Duffing Holmes (-)', style=sty)
LENJON=wid.Button(description='Lennard Jones', style=sty)
MORSE=wid.Button(description='Morse', style=sty)
POSTEL=wid.Button(description='Pöschl–Teller', style=sty)
TRIANG=wid.Button(description='Poço Triangular', style=sty)
CAMPMENOS=wid.Button(description='Campo Uniforme (-)', style=sty)
CAMPMAIS=wid.Button(description='Campo Uniforme (+)', style=sty)

################################ON_EVENTS###############################
DIAG.on_click(diagg)
OHS.on_click(potlist)
POCO.on_click(potlist)
DUFHOLMES.on_click(potlist)
MDUFHOLMES.on_click(potlist)
LENJON.on_click(potlist)
MORSE.on_click(potlist)
POSTEL.on_click(potlist)
TRIANG.on_click(potlist)
CAMPMAIS.on_click(potlist)
CAMPMENOS.on_click(potlist)


################################BOXES###############################
POTSBUT=wid.GridBox([POCO,TRIANG,CAMPMAIS,CAMPMENOS,OHS,DUFHOLMES,
                MDUFHOLMES,LENJON,MORSE,POSTEL],
    layout=wid.Layout(grid_template_columns="repeat(auto-fill, 180px)"))
POTLIST=wid.Accordion([POTSBUT])
INFODIAG=wid.VBox([info,DIAG])
INFO=wid.Accordion([INFODIAG])
POTLIST.set_title(0,'Potenciais pré definidos')
INFO.set_title(0,'Diagonalizar/Informações')
SYSTBOX=wid.VBox([POTLIST,SYST,INFO])

################################UI################################
SYST.children[3].description=('Gerar')
PLOT.children[8].description=('Plotar')
PLOT.children[8].disabled=True
INFO.selected_index=None
################################TABS################################
tab=wid.Tab(children=[SYSTBOX,PLOT])
tab.set_title(0, 'Gerar o Sistema')
tab.set_title(1, 'Funções de Onda')
display(tab)

***
# Unidades atômicas de Rydberg
Tabela de conversão de unidades entre o sistema de Rydberg e o MKSA, que é a base constante do SI. Cheque a fonte original mais abaixo para um tabela mais completa em inglês.

Fonte da tabela: [(Departamento de Física, Universidade Rei Fahd de Petróleos e Minerais)](https://faculty.kfupm.edu.sa/phys/aanaqvi/rydberg.pdf)

|Dimensão |  Sistema de Unidades Rydberg | MKSA (base do SI) | Outras equivalências|
| :- |  :-: | :-: | :-:|
|Comprimento | $$1=a_0$$ | $$5.2917725 \times 10^{-11} $$ | $$\approx 0.5291772 \text{ Å}$$|
|Massa| $$1=2m_e$$ | $$1.8218779\times 10^{-30} \text{ kg} $$| - |
|Tempo |$$1$$| $$1 4.8377687 \times 10^{-17} \text{ s}$$ | $$=1/20 \text{ fs}$$ |
|Frequência | $$1$$ | $$ 2.0670687 \times 10^{16} \text{ Hz}$$ | - |
|Velocidade| $$1$$ | $$1.0938457 \times 10^{6} \text{ m/s}$$| -|
|Momento| $$1$$ | $$1.9928534 \times 10^{-24} \text{ kg.m/s}$$| - |
|Energia| $$1=E_{Ryd}=e^2/(2a_0)$$ | $$2.1798741 \times 10 ^{-18} \text{ J}$$| $$3.605698 \text{ eV}$$|
|Carga | $$1=e/\sqrt{2}$$ | $$1.1329105\times 10^{-19} \text{ C}$$| - |
|Campo Elétrico |$$1$$| $$1 3.6360903 \times 10^{11} \text{ V/m}$$|-|
|Potencial | $$1$$ | $$19.241363 \text{ V}$$|-|

|Constantes Úteis | Símbolo |Sistema de Unidades Rydberg | MKSA (base do SI) |
|:-|:-:|:-:|:-|
|Constante de Planck reduzida | $$\hbar$$| $$1$$| $$1.05457266\times 10 ^{-34} \text{ J.s}$$|
|$h= 2\pi\hbar$| $$h$$ | $$2\pi$$ | $$6.6260755\times 10^{-34} \text{ J.s}$$|
|Carga do elétron | $$e$$ | $$\sqrt{2}$$| $$1.60217733\times 10 ^{-19} \text{ C}$$|
|Raio de Bohr|$$a_0$$| $$1=\hbar^2/(me^2)$$| $$5.29177249\times 10^{-11} \text{ m}$$|
|Energia de Hartree| $$E_h$$|$$2=e^2/(a_0)$$ | $$4.3597482\times 10^{-18} \text{ J}$$|
|Constante de estrutura fina| $$\alpha$$| $$e^2/(\hbar c)=1/137.0359895 $$| - |
|Velocidade da luz | $$c$$ | $$2/\alpha \approx 274$$ | $$2.99792458\times 10^{8} \text{ m/s}$$|
|Permeabilidade do vácuo| $$\mu_0$$ | $$\pi\alpha^2\approx 1.7 \times 10^{-4}$$| $$4\pi \times 10^{-7} \text{ H/m}$$|
|Permissividade do vácuo| $$\epsilon_0$$| $$1/(4\pi)\approx 0.08$$| $$8.854187817\times 10^{-12} \text{ F/m}$$|
|Massa do elétron| $$m_e$$| $$1/2$$ |$$9.1093897 \times 10^{-31} \text{ kg}$$|
|Massa do próton| $$m_p$$ | $$918.07638$$ | $$1.6726231\times 10^{-27} kg$$|

# Modelo Matemático
<details><summary>Clique aqui para mostrar</summary>
Escrevemos a equação fazendo $\hbar^2 /(2m)=1$, já que utilizaremos o sistema de unidades atômicas de Rydberg:
$$-\frac{\partial^2 \Psi}{\partial x^2}+V(x)=E\Psi$$

Discretizamos as derivadas parciais utilizando o método de diferenças finitas, fazendo a primeira derivada pela direita e a segunda pela esquerda para diminuir o erro:

$$\frac{\partial \Psi}{\partial x} \approx \frac{\Psi(x+\Delta x)-\Psi(x)}{\Delta x}$$

$$\frac{\partial \Psi}{\partial x} \approx \frac{\Psi(x)-\Psi(x-\Delta x)}{\Delta x}$$

Utilizando a notação com índices inteiros $[\Psi(x-\Delta x), \Psi(x), \Psi(x+\Delta x)]\rightarrow [\Psi_{i-1},\Psi_i, \Psi_{i+1}]$  e aplicando as derivadas acima, a equação original fica:

$$\frac{-\left(\Psi_{i-1}-2\Psi_{i}+\Psi_{i+1}\right)}{\Delta x^2}+V_i \Psi_i=E\Psi_i$$

Definindo $a={-1}/{\Delta x ^2}$:
$$a\left(\Psi_{i-1}-2\Psi_{i}+\Psi_{i+1}\right)+V_i \Psi_i=E\Psi_i$$
 
Suponha um vetor $\Psi$ contendo todos os valores discretos de uma função de onda unidimensional representada por $N$ pontos com uma indexação de que vai $i=1$ até $i=N$, ou seja:

$$\Psi=\begin{bmatrix}\Psi_{1}\\\Psi_{2}\\\Psi_3\\\vdots \\ \Psi_{N}\end{bmatrix}$$

Rearrajando as equações de uma forma conveniente, ficamos com o seguinte sistema:
 
$$\begin{matrix}
a\Psi_{0}&+& (V_1-2a)\Psi_{1}&+& a\Psi_{2}  &= & E\Psi_1 \\ 
a \Psi_{1}&+&(V_2-2a)\Psi_{2}&+&a\Psi_{3}& = & E\Psi_2 \\ 
a \Psi_{2}&+&(V_3-2a)\Psi_{3}&+&a\Psi_{4}& = & E\Psi_3 \\ 
\vdots & +& \vdots&+& \vdots & = &\vdots\\ 
a\Psi_{N-1}&+&(V_{N}-2a)\Psi_{N}&+&a\Psi_{N+1}& = & E\Psi_{N}
\end{matrix}$$

Note que na primeira e última equações do sistema acima, do lado esquerdo da igualdade, temos os termos $\Psi_0$ e $\Psi_{N+1}$, que não correspondem a nenhuma componente do nosso vetor da função de onda discreta $\Psi$. Nós iremos utilizá-los para definir as condições de contorno (o valor da função de onda nas bordas do sistema).

>## Condições de contorno
>### Fronteiras Fixas (poço infinito)
>Nessa condição forçamos a função de onda a ser 0 nas bordas, ou seja
>$$\Psi_0=\Psi_{N+1}=0$$
>### Condição de contorno periódica
>Nessa condição forçamos a função de onda no último ponto a ser igual ao valor do primeiro ponto no grid. 
>$$\Psi_0=\Psi_{N+1}=\Psi_{1}=\Psi_{N}$$
## Equação matricial

Aplicando a condição de contorno do poço infinito, temos:
$$\begin{matrix}
0&+& \color{blue}{(V_1-2a)}\Psi_{1}&+& \color{blue}a \Psi_{2}  &= & E\Psi_1 \\ 
\color{red}a \Psi_{1}&+&\color{red}{(V_2-2a)}\Psi_{2}&+&\color{red}a\Psi_{3}& = & E\Psi_2 \\ 
\color{orange}a \Psi_{2}&+&\color{orange}{(V_3-2a)}\Psi_{3}&+&\color{orange}a \Psi_{4}& = & E\Psi_3 \\ 
\vdots & +& \vdots&+& \vdots & = &\vdots\\ 
\color{purple}a\Psi_{N-2}&+&\color{purple}{(V_{N-1}-2a)} \Psi_{N-1}& +&\color{purple}a \Psi_{N}&= & E\Psi_{N-1}\\
\color{green}a\Psi_{N-1}&+&\color{green}{(V_{N}-2a)} \Psi_{N}&+&0& = & E\Psi_{N}
\end{matrix}$$

Esse sistema pode ser reescrito como uma matriz do tipo:
$$\begin{bmatrix}  \color{blue}{(V_1-2a)} & \color{blue}a & 0  & 0 & 0 & 0\\
\color{red}a & \color{red}{(V_2-2a)} & \color{red}a  & 0 & 0 &0 \\
0&\color{orange}a & \color{orange}{(V_3-2a)} &  \color{orange}a  & 0  &0  \\
 0 & 0 & \ddots & \ddots & \ddots & 0 \\
0& 0 & 0  & \color{purple}a & \color{purple}{(V_{N-1}-2a)}& \color{purple}a\\
0& 0 & 0 & 0 & \color{green}a & \color{green}{(V_{N}-2a)} 
\end{bmatrix} 
\begin{bmatrix}\Psi_{1}\\\Psi_2\\\Psi_3\\\vdots \\\Psi_{N-1}\\ \Psi_{N}\end{bmatrix}=E\begin{bmatrix}\Psi_{1}\\\Psi_2\\\Psi_3\\\vdots \\ \Psi_{N-1}\\\Psi_{N}\end{bmatrix}$$

Isso é equivalente a um problema de autovetores e autovalores, onde a matriz quadrada de ordem $N$ acima é o operador Hamiltoniano $\hat{H}$:

$$\hat{H}\Psi=E\Psi$$

Podemos diagonalizá-lo para obter os autovalores e seus respectivos autovetores. Os autovetores serão vetores $\Psi_n$ com $N$ componentes representando as respectivas funções de ondas dos estados $n$, e os autovalores $E$ serão as energias correspondentes aos respectivos estados. 
</details>
